In [1]:
!pip install langchain_openai
!pip install langchain_core
!pip install langchain_community
!pip install langserve
!pip install pineconedb
!pip install openai pinecone-client langchain
!pip install deepeval

ERROR: Could not find a version that satisfies the requirement pineconedb (from versions: none)
ERROR: No matching distribution found for pineconedb


In [2]:
import pandas as pd
import numpy as np
import langchain_openai as lo

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
a='/content/drive/MyDrive/data.csv'
docs=pd.read_csv(a)
docs.head()

,Unnamed: 0,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription
0,0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified
1,1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti..."
2,2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...
3,3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res..."
4,4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen


In [19]:
import os
os.environ['OPENAI_API_KEY']="sk-proj-..."


In [6]:
import langchain_community.vectorstores as vectorstores

In [5]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="...")


In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
embed = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    openai_api_key="sk-proj-..."
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [26]:
for index, row in docs.iterrows():
  string='Patient '
  if row['Age'] not in [' NA','NA',' Not specified']:
    string+=f"of Age {row['Age']},"
  if row['Gender']not in [' NA','NA',' Not specified']:
    string+=f"of Gender {row['Gender']},"
  string+=f"is having {row['Symptoms']}"
  docs.loc[index,'Summary']=string
docs.head()

,Unnamed: 0,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription,Summary
0,0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified,Patient is having Severe sharp pain in the up...
1,1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti...","Patient of Age 30,of Gender Female,is having..."
2,2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...,"Patient is having Intense, constant sharp pai..."
3,3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res...",Patient is having Lower back pain for two wee...
4,4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen,"Patient of Age 5,of Gender Female,is having ..."


In [10]:
docs['VectorSummary']=docs['Summary'].apply(lambda x: embed.embed_query(x))
docs.head()

,Unnamed: 0,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription,Summary,VectorSummary
0,0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified,Patient is having Severe sharp pain in the up...,"[0.0035097879048491916, -0.02304348576684826, ..."
1,1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti...","Patient of Age 30,of Gender Female,is having...","[-0.004388182736946255, -0.006196587596492887,..."
2,2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...,"Patient is having Intense, constant sharp pai...","[-0.013259365625580756, -0.0014831667639806582..."
3,3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res...",Patient is having Lower back pain for two wee...,"[0.014604454649253994, 0.003056069326240236, 0..."
4,4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen,"Patient of Age 5,of Gender Female,is having ...","[-0.010854082485243253, 0.010485031003581741, ..."


In [ ]:
docs['Disease'].fillna('Unknown', inplace=True)
docs['Diagnosis'].fillna('Unknown', inplace=True)
docs['Prescription'].fillna('Unknown', inplace=True)

In [ ]:
#docs.drop(columns=['Age','Gender'],inplace=True)
docs['Unnamed: 0']=docs['Unnamed: 0'].apply(lambda x: str(x))
docs.head()

,Unnamed: 0,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription,Summary,VectorSummary
0,0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified,Patient is having Severe sharp pain in the up...,"[0.0035097879048491916, -0.02304348576684826, ..."
1,1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti...","Patient of Age 30,of Gender Female,is having...","[-0.004388182736946255, -0.006196587596492887,..."
2,2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...,"Patient is having Intense, constant sharp pai...","[-0.013259365625580756, -0.0014831667639806582..."
3,3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res...",Patient is having Lower back pain for two wee...,"[0.014604454649253994, 0.003056069326240236, 0..."
4,4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen,"Patient of Age 5,of Gender Female,is having ...","[-0.010854082485243253, 0.010485031003581741, ..."


In [ ]:
type(docs['Unnamed: 0'][0])

str

In [ ]:
pc.create_index(
    name="disease-symptoms-gpt-4",
    dimension=len(docs['VectorSummary'][0]), # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [7]:
index = pc.Index("disease-symptoms-gpt-4")
# wait a moment for connection

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 272}},
 'total_vector_count': 272}

In [ ]:
docs.head()
for i,j in docs.iterrows():
  index.upsert(vectors=[(j['Unnamed: 0'],j['VectorSummary'],{'text':j['Summary'],'Disease':j['Disease'],'Diagnosis':j['Diagnosis'],'Prescription':j['Prescription']})])
index.describe_index_stats()

In [12]:
from langchain.vectorstores import Pinecone
# switch back to normal index for langchain

vectorstore = Pinecone(index, embed.embed_query, "text")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Semantic Search



In [13]:
query = "pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite"


vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(metadata={'Diagnosis': ' Abdominal pain, likely related to gallbladder issues', 'Disease': ' Possible gallbladder disease', 'Prescription': ' Not specified'}, page_content='Patient is having  Severe sharp pain in the upper right abdomen, nausea, decreased energy levels, reduced appetite'),
 Document(metadata={'Diagnosis': ' Not specified', 'Disease': ' Not specified', 'Prescription': ' Not specified'}, page_content='Patient is having  Pain in the right side of the belly, aching pain, constant pain for the past 5-6 hours, nausea, fatigue, reduced appetite'),
 Document(metadata={'Diagnosis': ' NA', 'Disease': ' NA', 'Prescription': ' NA'}, page_content='Patient is having  Abdominal pain near the right hip, crampy feeling, constipation, nausea, vomiting, feeling hot (possible fever)')]

RAG Retriver

In [14]:
from langchain.vectorstores import Pinecone as LangChainPinecone
from langchain.prompts import PromptTemplate
from openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_core.vectorstores import VectorStoreRetriever

prompt_template='''Accept the user’s symptoms as input and provide probable diseases, diagnoses and prescription using only the information stored in the vector database. If the information is not available in the database, politely inform the user that the data is insufficient to provide a diagnosis.
    Text:
    {context}'''
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context"]
)
# Create a RetrievalQA chain
retriever = VectorStoreRetriever(vectorstore=vectorstore)
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(api_key="sk-proj-...",
                   model_name='gpt-4o',
                   temperature=0.0),
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [15]:
q=' pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite'
answer = qa_chain.run(query=q)
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Based on the symptoms provided, here are the probable diseases, diagnoses, and potential prescriptions:

1. **Patient with severe sharp pain in the upper right abdomen, nausea, decreased energy levels, and reduced appetite:**
   - **Probable Disease:** Gallstones or Cholecystitis
   - **Diagnosis:** The symptoms suggest a possible issue with the gallbladder, such as gallstones or inflammation of the gallbladder (cholecystitis).
   - **Prescription:** 
     - Pain management with NSAIDs or acetaminophen.
     - Antiemetics for nausea.
     - If cholecystitis is confirmed, antibiotics may be prescribed.
     - Surgical consultation for possible gallbladder removal if gallstones are confirmed.

2. **Patient with pain in the right side of the belly, aching pain, constant pain for the past 5-6 hours, nausea, fatigue, and reduced appetite:**
   - **Probable Disease:** Appendicitis
   - **Diagnosis:** The symptoms are indicative of appendicitis, especially with the constant pain and location.

In [20]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric

test_case = LLMTestCase(input=q, actual_output=answer, expected_output=)
relevancy_metric = AnswerRelevancyMetric(threshold=0.5)

relevancy_metric.measure(test_case)
print(relevancy_metric.score, relevancy_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0 The score is 1.00 because the response perfectly addresses the symptoms mentioned without any irrelevant information. Great job!


In [23]:
relevancy_metric._calculate_score()

1.0